In [ ]:
#module imports
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow_probability as tfp
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [ ]:
import rpy2

## (Activate R Environment)

In [ ]:
%reload_ext rpy2.ipython

# Directories

In [ ]:
#Set working director
os.chdir('C:/Users/matth/OneDrive/Durham/Masters/Diss')

In [ ]:
# Create 'plots' directory for saving plots
plots_dir = os.path.join(os.getcwd(), "plots")
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Create 'models' directory for saving models
models_dir = os.path.join(os.getcwd(), "models")
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# R Libraries

In [ ]:
%%R
library(data.table)
library(ggplot2)
library(ggnewscale)
library(tidyverse)
library(rgdal)
library(raster)
library(scoringRules)
library(scoringutils)
library(raster)

options(scipen = 6000)

# Elevation Data and Plot

In [ ]:
%%R
#Load in Elevation raster to R and convert to dataframe
elevation_r <- raster("C:/Users/matth/OneDrive/Durham/Masters/Diss/Data/DEM.tif")
elevimage <- as.data.table(raster::as.data.frame(elevation_r, xy = TRUE, na.rm = TRUE))

In [ ]:
%%R
###Plot elevation raster
ggplot(elevimage) + geom_raster(aes(x = x, y = y, fill = DEM)) + theme_bw() +
  scale_fill_viridis_c(option = "cividis", begin = 0.2, end = 1, name = 'Elevation, m') + coord_equal() +
  theme(legend.justification = c(1, 1), legend.position = c(0.35, 0.99), legend.background=element_blank(),
        axis.text.y = element_text(angle = 90, vjust = 0.5, hjust=0.5)) + labs(x = "Easting (BNG)", y = "Northing (BNG)") +
  scale_x_continuous(breaks=seq(250000,500000,by=50000),limits = c(275000,475000)) + scale_y_continuous(breaks=seq(400000,700000,by=50000),limits = c(450000,675000))
ggsave(paste0("plots/NEelevation.png"), width = 68, height = 100, units = "mm", type = "cairo", dpi = 300, scale = 1.375)

# Gravity Anomaly Data and Plot

In [ ]:
%%R
#Load in Gravity Survey data into R and convert to dataframe
gravity_r <- raster('C:/Users/matth/OneDrive/Durham/Masters/Diss/Data/Grav.tif')
gravimage<-as.data.table(raster::as.data.frame(gravity_r, xy = TRUE, na.rm = TRUE))

In [ ]:
%%R
###Plot Bouguer Anomaly raster
ggplot(gravimage) + geom_raster(aes(x = x, y = y, fill = Grav)) + theme_bw() +
  scale_fill_viridis_c(option = "cividis", begin = 0.2, end = 1, name = 'Bouguer Anomaly, mGal') + coord_equal() +
  theme(legend.justification = c(1, 1), legend.position = c(0.60, 0.99), legend.background=element_blank(),
        axis.text.y = element_text(angle = 90, vjust = 0.5, hjust=0.5)) + labs(x = "Easting (BNG)", y = "Northing (BNG)") +
  scale_x_continuous(breaks=seq(250000,500000,by=50000),limits = c(275000,475000)) + scale_y_continuous(breaks=seq(400000,700000,by=50000),limits = c(440000,685000))
ggsave(paste0("plots/NEGravity.png"), width = 68, height = 100, units = "mm", type = "cairo", dpi = 300, scale = 1.375)

# EVI Data and Plot

In [ ]:
%%R
#Load in EVI data into R and convert to dataframe
evi_r <- raster('C:/Users/matth/OneDrive/Durham/Masters/Diss/Data/VI.tif')
eviimage<-as.data.table(raster::as.data.frame(evi_r, xy = TRUE, na.rm = TRUE))

In [ ]:
%%R
###Plot EVI raster
ggplot(eviimage) + geom_raster(aes(x = x, y = y, fill = Band_1/10000)) + theme_bw() +
  scale_fill_viridis_c(option = "cividis", begin = 0.2, end = 1, name = 'EVI ') + coord_equal() +
  theme(legend.justification = c(1, 1), legend.position = c(0.25, 0.99), legend.background=element_blank(),
        axis.text.y = element_text(angle = 90, vjust = 0.5, hjust=0.5)) + labs(x = "Easting (BNG)", y = "Northing (BNG)") +
  scale_x_continuous(breaks=seq(250000,500000,by=50000),limits = c(275000,475000)) + scale_y_continuous(breaks=seq(400000,700000,by=50000),limits = c(440000,685000))
ggsave(paste0("plots/NEEVI.png"), width = 68, height = 100, units = "mm", type = "cairo", dpi = 300, scale = 1.375)

# Mineral Data and Plots

In [ ]:
#Read in mineral data
Chemical=pd.read_csv('C:/Users/matth/OneDrive/Durham/Masters/Diss/Data/Points.csv')

In [ ]:
# Plot mineral data
plt.figure(figsize=(6.8, 10))
plt.scatter(Chemical['X_COORD'], Chemical['Y_COORD'], color='black', marker='o', s=1, alpha=0.2)
plt.xlim(275000, 475000)
plt.ylim(450000, 675000)
plt.xlabel('Easting (BNG)')
plt.ylabel('Northing (BNG)')
plt.yticks(range(450000,700000,50000))
plt.xticks(range(300000,500000,50000))
plt.title('UK Geochem Plot')
plt.legend().set_visible(False)
plt.gca().set_aspect('equal', adjustable='box')

# Save the plot
plt.savefig("plots/UKRSSPoints.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Choose an element name to continue with, out of available names:
elem = "Ni_DCOES"
elemname = "Nickel"

In [ ]:
#Compare histograms to see whether a log transformation is appropriate to develop a Gaussian distribution. 0.1 is added to each value to remove 0 values as this would error under logarithmic transformation - this can be removed after analysis.
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.hist(Chemical[elem])
ax2.hist(np.log(Chemical[elem]+0.1))
ax1.title.set_text('Untransformed')
ax2.title.set_text('Logarithmic')
plt.show()

In [ ]:
# Should a log transformation be used?
logtrans = True

# Plot Mineral Data Coloured by Mineral Concentration

In [ ]:
# Create a color scale
cmap = plt.get_cmap('inferno')
if logtrans is True:
    color_values = np.log(Chemical[elem] + 0.1)
else:
    color_values = Chemical[elem]

# Create the scatter plot
plt.figure(figsize=(7,5))
plt.scatter(Chemical['X_COORD'], Chemical['Y_COORD'], c=color_values, cmap=cmap, s=5)
if logtrans:
    norm = plt.Normalize(np.log(Chemical[elem] + 0.1).min(), np.log(Chemical[elem] + 0.1).max())
else:
    norm = plt.Normalize(Chemical[elem].min(), Chemical[elem].max())
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca())
cbar.set_label(f"log({elemname} Concentration)" if logtrans else f"{elemname} Concentration")
plt.xlabel('Easting (BNG)')
plt.ylabel('Northing (BNG)')
plt.yticks(range(450000,700000,50000))
plt.xticks(range(300000,500000,50000))
plt.title('River Sediment Sample Points')
plt.legend().set_visible(False)  # Legend not shown in this case

# Show the plot and save it
plt.savefig(f'plots/{elemname}_Points.png')

# Extract Location Data, Elevation, Bouguer Anomaly and EVI Values at Each Sample Point

In [ ]:
%%R -i Chemical

#Load Mineral data into R, to allow for bilinear sample extraction from rasters
Chemical <- as.data.table(Chemical)

In [ ]:
%%R -i elem -o Chemical

#Extract Elevation, Bouguer Anomaly and EVI at each sample point, and remove NA's
Chemical[, elevation := raster::extract(elevation_r, Chemical[, c("X_COORD", "Y_COORD"), with = FALSE], method = "bilinear"), ]
Chemical[, gravity := raster::extract(gravity_r, Chemical[, c("X_COORD", "Y_COORD"), with = FALSE], method = "bilinear"), ]
Chemical[, evi := raster::extract(evi_r, Chemical[, c("X_COORD", "Y_COORD"), with = FALSE], method = "bilinear"), ]

Chemical <- Chemical[!is.na(Chemical[,get(elem)]) & !is.na(Chemical$elevation) & !is.na(Chemical$gravity) & !is.na(Chemical$evi)]

# Image classification

In [ ]:
#Set the pixel size and image size of the images for use in the convolutional networks
imagedim = 32
imageres = 250

In [ ]:
%%R -i imagedim,imageres,Chemical

### Extract sample-centred images, for use in feature learning ####
#Create array of empty images
imgs_elev <- array(dim = c(nrow(Chemical), imagedim, imagedim))
imgs_grav <- array(dim = c(nrow(Chemical), imagedim, imagedim))
imgs_evi <- array(dim = c(nrow(Chemical), imagedim, imagedim))

#Creat image reference for use in loop
cells <- as.data.table(expand.grid(x = seq.int(from = imageres/2, by = imageres, length.out = imagedim)-(imageres*imagedim)/2, 
                                   y = seq.int(from = imageres/2, by = imageres, length.out = imagedim)-(imageres*imagedim)/2))
cells[ ,coordx := rep(1:imagedim, imagedim),]
cells[ ,coordy := rep(1:imagedim, each = imagedim),]

#Loop through each pixel and extract values, for each sample point
for(xmeter in unique(cells$coordx)){
  for(ymeter in unique(cells$coordy)){
    imgs_elev[,xmeter,ymeter] <- extract(elevation_r, method = "bilinear",
                                    cbind(Chemical$X_COORD + cells[coordx == xmeter & coordy == ymeter]$x, Chemical$Y_COORD + cells[coordx == xmeter & coordy == ymeter]$y)) 
    imgs_grav[,xmeter,ymeter] <- extract(gravity_r, method = "bilinear",
                                    cbind(Chemical$X_COORD + cells[coordx == xmeter & coordy == ymeter]$x, Chemical$Y_COORD + cells[coordx == xmeter & coordy == ymeter]$y))
    imgs_evi[,xmeter,ymeter] <- extract(evi_r, method = "bilinear",
                                    cbind(Chemical$X_COORD + cells[coordx == xmeter & coordy == ymeter]$x, Chemical$Y_COORD + cells[coordx == xmeter & coordy == ymeter]$y))
  }
}

#Set NA values to 0
imgs_elev[is.na(imgs_elev)] = 0
imgs_grav[is.na(imgs_grav)] = 0
imgs_evi[is.na(imgs_evi)] = 0

In [ ]:
%%R -o elev_ann,grav_ann,evi_ann

#Sample centre the values of each image
elev_ann = imgs_elev - Chemical$elevation
grav_ann = imgs_grav - Chemical$gravity
evi_ann = imgs_evi - Chemical$evi

# Location data

In [ ]:
# Extract easting, northing, elevation, bouguer anomaly and evi as location variables
loc = Chemical.loc[:,["X_COORD", "Y_COORD", "elevation","gravity",'evi']]

# Creating Test Train Val Splits

In [ ]:
#Create train, validation and test splits
train, testval = train_test_split(np.arange(Chemical.shape[0]), train_size=0.7, random_state = 2023)
val, test = train_test_split(testval, train_size=0.5, random_state = 2001)

In [ ]:
#Split each of the inputs into train, validation and test
loc_train = loc.iloc[train]
loc_val = loc.iloc[val]
loc_test = loc.iloc[test]

In [ ]:
elev_ann_train = elev_ann[train,]
elev_ann_val = elev_ann[val,]
elev_ann_test = elev_ann[test,]

In [ ]:
grav_ann_train = grav_ann[train,]
grav_ann_val = grav_ann[val,]
grav_ann_test = grav_ann[test,]

In [ ]:
evi_ann_train = evi_ann[train,]
evi_ann_val = evi_ann[val,]
evi_ann_test = evi_ann[test,]

# Normalising

In [ ]:
#Normalise each input by the TRAIN standard deviation, to prevent data leakage
elev_sd = elev_ann_train.std()

elev_ann_train = elev_ann_train/elev_sd
elev_ann_val = elev_ann_val/elev_sd
elev_ann_test = elev_ann_test/elev_sd

In [ ]:
grav_sd = grav_ann_train.std()

grav_ann_train = grav_ann_train/grav_sd
grav_ann_val = grav_ann_val/grav_sd
grav_ann_test = grav_ann_test/grav_sd

In [ ]:
evi_sd = evi_ann_train.std()

evi_ann_train = evi_ann_train/evi_sd
evi_ann_val = evi_ann_val/evi_sd
evi_ann_test = evi_ann_test/evi_sd

In [ ]:
locmean = loc_train.mean(axis=0)
locsd = loc_train.std(axis=0)

loc_train = (loc_train-locmean)/locsd
loc_val = (loc_val-locmean)/locsd
loc_test = (loc_test-locmean)/locsd

# Inputs and Outputs

In [ ]:
# Finally combine the independent variables/inputs
x_train = [elev_ann_train,grav_ann_train,evi_ann_train,loc_train]
x_val = [elev_ann_val,grav_ann_val,evi_ann_val,loc_val]
x_test = [elev_ann_test,grav_ann_test,evi_ann_test,loc_test]

In [ ]:
# Create the dependant variables/expected outputs
if(logtrans == True):
    y_train = np.log(Chemical.loc[:, elem]+0.1)[train]
    y_val = np.log(Chemical.loc[:, elem]+0.1)[val]
    y_test = np.log(Chemical.loc[:, elem]+0.1)[test]
else:
    y_train = Chemical.loc[:, elem][train]
    y_val = Chemical.loc[:, elem][val]
    y_test = Chemical.loc[:, elem][test]

# Construct the deep learning neural network

In [ ]:
#Set tuned dropout rate for convolutional and fully connected layers
dropratespat=0.65
dropratedense=0.3

In [ ]:
# Convolutional stack for elevation images:
conv_input_elev = layers.Input(shape=(imagedim, imagedim, 1), name='conv_input_elev')

conv_output_elev = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=3)(conv_input_elev)
conv_output_elev = layers.Activation('relu')(conv_output_elev)
conv_output_elev = layers.SpatialDropout2D(rate=dropratespat)(conv_output_elev)

conv_output_elev = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_elev)
conv_output_elev = layers.Activation('relu')(conv_output_elev)
conv_output_elev = layers.SpatialDropout2D(rate=dropratespat)(conv_output_elev)

conv_output_elev = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_elev)
conv_output_elev = layers.Activation('relu')(conv_output_elev)
conv_output_elev = layers.SpatialDropout2D(rate=dropratespat)(conv_output_elev)

conv_output_elev = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_elev)
conv_output_elev = layers.Activation('relu')(conv_output_elev)
conv_output_elev = layers.SpatialDropout2D(rate=dropratespat)(conv_output_elev)

conv_output_elev = layers.GlobalAveragePooling2D()(conv_output_elev)
conv_output_elev = layers.Flatten()(conv_output_elev)

In [ ]:
# Convolutional stack for bouguer anomaly images:
conv_input_grav = layers.Input(shape=(imagedim, imagedim, 1), name='conv_input_grav')

conv_output_grav = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=3)(conv_input_grav)
conv_output_grav = layers.Activation('relu')(conv_output_grav)
conv_output_grav = layers.SpatialDropout2D(rate=dropratespat)(conv_output_grav)

conv_output_grav = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_grav)
conv_output_grav = layers.Activation('relu')(conv_output_grav)
conv_output_grav = layers.SpatialDropout2D(rate=dropratespat)(conv_output_grav)

conv_output_grav = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_grav)
conv_output_grav = layers.Activation('relu')(conv_output_grav)
conv_output_grav = layers.SpatialDropout2D(rate=dropratespat)(conv_output_grav)

conv_output_grav = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_grav)
conv_output_grav = layers.Activation('relu')(conv_output_grav)
conv_output_grav = layers.SpatialDropout2D(rate=dropratespat)(conv_output_grav)

conv_output_grav = layers.GlobalAveragePooling2D()(conv_output_grav)
conv_output_grav = layers.Flatten()(conv_output_grav)

In [ ]:
# Convolutional stack for EVI images:
conv_input_evi = layers.Input(shape=(imagedim, imagedim, 1), name='conv_input_evi')

conv_output_evi = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=3)(conv_input_evi)
conv_output_evi = layers.Activation('relu')(conv_output_evi)
conv_output_evi = layers.SpatialDropout2D(rate=dropratespat)(conv_output_evi)

conv_output_evi = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_evi)
conv_output_evi = layers.Activation('relu')(conv_output_evi)
conv_output_evi = layers.SpatialDropout2D(rate=dropratespat)(conv_output_evi)

conv_output_evi = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_evi)
conv_output_evi = layers.Activation('relu')(conv_output_evi)
conv_output_evi = layers.SpatialDropout2D(rate=dropratespat)(conv_output_evi)

conv_output_evi = layers.Conv2D(128, kernel_size=(3, 3), dilation_rate=1, strides=1)(conv_output_evi)
conv_output_evi = layers.Activation('relu')(conv_output_evi)
conv_output_evi = layers.SpatialDropout2D(rate=dropratespat)(conv_output_evi)

conv_output_evi = layers.GlobalAveragePooling2D()(conv_output_evi)
conv_output_evi = layers.Flatten()(conv_output_evi)

In [ ]:
# Auxiliary input for locational and auxiliary data:
auxiliary_input = layers.Input(shape=(5,), name='aux_input')

auxiliary_output = layers.Dense(1920)(auxiliary_input)
auxiliary_output = layers.Activation('relu')(auxiliary_output)
auxiliary_output = layers.Dropout(rate=dropratedense)(auxiliary_output)
auxiliary_output = layers.Flatten()(auxiliary_output)

In [ ]:
# Concated inputs and final output:
concatenated_output = layers.concatenate([conv_output_elev,conv_output_grav,conv_output_evi,auxiliary_output])
main_output = layers.Dense(1024)(concatenated_output)
main_output = layers.Activation('relu')(main_output)
main_output = layers.Dropout(rate=dropratedense)(main_output)
main_output = layers.Dense(256)(main_output)
main_output = layers.Activation('relu')(main_output)
main_output = layers.Dropout(rate=dropratedense)(main_output)
main_output = layers.Dense(units=2, activation='linear', name='dist_param')(main_output)
main_output = tfp.layers.DistributionLambda(lambda x: tfp.distributions.Normal(
    loc=x[..., :1],
    #Softplus link function to force the variance to be positive
    scale=1e-3 + tf.math.softplus(0.1 * x[..., 1:])
))(main_output)

In [ ]:
#Combining the final model
np.random.seed(1)
tf.random.set_seed(1)
model = Model(inputs=[conv_input_elev, conv_input_grav, conv_input_evi, auxiliary_input], outputs=main_output)

In [ ]:
#Summary of the mode
model.summary()

In [ ]:
#Define Negative Log Likelihood loss function and set the optimiser
def negloglik(y, model):
    return -model.log_prob(y)
opt = tf.keras.optimizers.Adam(learning_rate=0.002, decay=1e-6)

In [ ]:
#Compile the model
model.compile(loss=negloglik, optimizer=opt)

# Train the Model

In [ ]:
# Set the Batch Size and Number of Epochs
batch_size = 2**12
epochs = 600


In [ ]:
#Fit the model
history = model.fit(x=x_train, y=y_train, batch_size=batch_size,epochs=epochs,validation_data=(x_val, y_val),shuffle=True,verbose=2,callbacks=[EarlyStopping(monitor="val_loss", patience=200),ModelCheckpoint(filepath="models/modelweights.hdf5",monitor="val_loss",save_best_only=True,save_weights_only=True)])
min_val_loss = min(history.history['val_loss'])

In [ ]:
#Show minimum validation loss, which will be the model used
print("Minimum Validation Loss:", {min_val_loss})

In [ ]:
# Show training history
trainhist = pd.DataFrame({'training': history.history['loss'], 'testing': history.history['val_loss']})
trainhist['epoch'] = np.arange(1, 601)
plt.figure(figsize=(10, 5))
sns.lineplot(data=pd.melt(trainhist, id_vars='epoch', value_name='NLL', var_name='dataset'), x='epoch', y='NLL', hue='dataset')
plt.ylim(0, np.quantile(trainhist['testing'], 0.999))
plt.xlabel('Epoch')
plt.ylabel('Negative Log Likelihood')
plt.savefig(f"plots/Training_{elemname}.png", dpi=800)
plt.show()

# Save as Mean Model

In [ ]:
#Load Model Weights from Lowest Validation Loss Model
model.load_weights("models/modelweights.hdf5")

In [ ]:
# Compile same model to save entire model with weights
meanmodel = Model(inputs=[model.input],outputs=[model.get_layer("dist_param").output])

meanmodel.save("models/meanmodel")
meanmodel = load_model("models/meanmodel")

tf.keras.backend.set_learning_phase(0)

# Evaluate the Model

In [ ]:
# Create dataframe of observed and predicted test values using model
holdout = pd.DataFrame({'obs': y_test.ravel(), 'preds': meanmodel.predict(x_test)[:,0].ravel()})

#Calculate R Squared and RMSE
r_squared = np.round(np.corrcoef(holdout['preds'], holdout['obs'])[0, 1] ** 2, 3)
rmse = np.round(np.sqrt(np.mean((holdout['preds'] - holdout['obs']) ** 2)), 3)

# Plot observed vs predicted
plt.figure(figsize=(8, 8))
plt.scatter
sns.scatterplot(data=holdout, x='obs', y='preds', alpha=0.1)
plt.plot(holdout['obs'], holdout['obs'], color='red', linestyle='--')
plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.title(f"R squared = {r_squared}, RMSE = {rmse}")
plt.savefig(f"plots/mean_holdout_{elemname}.png", dpi=800)
plt.show()

In [ ]:
# Print R Squared and RMSE
print(f"R squared = {r_squared}")
print(f"RMSE = {rmse}")

In [ ]:
# Create list of predictions
tf.keras.backend.set_learning_phase(1)
y_pred = meanmodel.predict(x_test, batch_size=batch_size)[:,0]

In [ ]:
%%R -i y_test,y_pred -o mean_testcrps
#Calculate CRPS score
testcrps <- crps_sample(y_test, drop(replicate(50,y_pred)))
mean_testcrps <- mean(testcrps)

In [ ]:
#Show CRPS score
print("Mean CRPS:", mean_testcrps)

In [ ]:
%%R -i y_test,y_pred,logtrans,elem,elemname
#Plot Probability Density
ggplot() +
  stat_density(data = data.frame(x = as.numeric(y_test), data = "Observed"), aes(x = x, col = data, linetype = data), geom = "line") +
  stat_density(data = data.frame(x = as.numeric(replicate(50, y_pred)), data = "Predicted"), 
               aes(x = x, col = data, linetype = data), geom = "line") +
  scale_x_continuous(limits = c(0,10)) + theme_bw() + theme(legend.justification = c(1,1), legend.position = c(0.95, 0.95), legend.title = element_blank()) +
  labs(x = if(logtrans == TRUE){paste0("log(", elem, ")")}else{paste(elem)},
       y = 'Density',
       subtitle = paste0("CRPS = ", round(mean(testcrps), 2)),
       tag = expression(bold("b")))
ggsave(paste0("plots/distribution_",elemname,".png"), width = 89, height = 89, units = "mm", type = "cairo", dpi = 300, scale = 1.375)

In [ ]:
%%R -i y_pred
#Test Quantiles
covtest <- melt(as.data.table(data.frame(y = y_test, x = drop(replicate(100, y_pred)))), id.vars = "y")
paste("95% =", round(nrow(covtest[y < quantile(value, 0.975) & y > quantile(value, 0.025)])/nrow(covtest), 3))

In [ ]:
%%R
paste("70% =", round(nrow(covtest[y < quantile(value, 0.85) & y > quantile(value, 0.15)])/nrow(covtest), 3))

In [ ]:
%%R
paste("50% =", round(nrow(covtest[y < quantile(value, 0.75) & y > quantile(value, 0.25)])/nrow(covtest), 3))

# Mean Map

In [ ]:
%%R -o predgrid
###Create Inputs for National Scale maps
#Start with auxiliary info
gbelevcoarse <- aggregate(elevation_r, fact=1)
predgrid <- as.data.table(raster::as.data.frame(gbelevcoarse, xy = TRUE, na.rm = TRUE))[x > 0 & y > 0]
setnames(predgrid, "DEM", "elevation")
predgrid$gravity <- extract(gravity_r,method='bilinear', cbind(predgrid$x,predgrid$y))
predgrid$evi <- extract(evi_r,method='bilinear', cbind(predgrid$x,predgrid$y))

In [ ]:
%%R -o predimgs_elevann,predimgs_gravann,predimgs_eviann
#Next get grid cell-centred images for the convolutional stack
predimgs_elev <- array(dim = c(nrow(predgrid), imagedim, imagedim))
predimgs_grav <- array(dim = c(nrow(predgrid), imagedim, imagedim))
predimgs_evi <- array(dim = c(nrow(predgrid), imagedim, imagedim))

for(xmeter in unique(cells$coordx)){
  for(ymeter in unique(cells$coordy)){
    predimgs_elev[,xmeter,ymeter] <- extract(elevation_r, method = "bilinear",
                                        cbind(predgrid$x + cells[coordx == xmeter & coordy == ymeter]$x, predgrid$y + cells[coordx == xmeter & coordy == ymeter]$y))
    predimgs_grav[,xmeter,ymeter] <- extract(gravity_r, method = "bilinear",
                                        cbind(predgrid$x + cells[coordx == xmeter & coordy == ymeter]$x, predgrid$y + cells[coordx == xmeter & coordy == ymeter]$y)) 
    predimgs_evi[,xmeter,ymeter] <- extract(evi_r, method = "bilinear",
                                        cbind(predgrid$x + cells[coordx == xmeter & coordy == ymeter]$x, predgrid$y + cells[coordx == xmeter & coordy == ymeter]$y)) 
  }
}

predimgs_elev[is.na(predimgs_elev)] = 0
predimgs_elevann <- predimgs_elev - predgrid$elevation

predimgs_grav[is.na(predimgs_grav)] = 0
predimgs_gravann <- predimgs_grav - predgrid$gravity

predimgs_evi[is.na(predimgs_evi)] = 0
predimgs_eviann <- predimgs_evi - predgrid$evi


In [ ]:
#Normalise the image arrays
predimgs_elevann = predimgs_elevann/elev_sd
predimgs_gravann = predimgs_gravann/grav_sd
predimgs_eviann = predimgs_eviann/evi_sd

In [ ]:
#Normalise the auxiliary data
predloc = predgrid[['x', 'y', 'elevation','gravity','evi']]
predloc=predloc.rename(columns={'x':'X_COORD','y':'Y_COORD'})

predloc_ann = (predloc-locmean)/locsd

In [ ]:
#Generate predictions for full study area for both mean and variance (aleatoric uncertainty)
pred = meanmodel.predict([predimgs_elevann,predimgs_gravann,predimgs_eviann,predloc_ann], batch_size=batch_size)
predgrid[elemname] = pred[:,0]
aleatoric = np.array(1e-3 + tf.math.softplus(0.1 * pred[:,1]))

#Generate mean raster
predraster = predgrid[['x','y',elemname]]

In [ ]:
%%R -i predraster,elemname,holdout

#Plot the mean map
ggplot(predraster) + geom_raster(aes(x = x, y = y, fill = get(elemname)), interpolate = FALSE) + theme_bw() + coord_equal() +
  scale_fill_viridis_c(option = "B", name = if(logtrans == TRUE){paste0("Predicted\nlog(",elemname," Concentration)")}else{paste("predicted\n", elemname, " Concentration")}) + labs(x = "Easting (metres BNG)", y = "Northing (metres BNG)") +
  theme(legend.justification = c(1, 1), legend.position = c(0.35, 0.99), legend.background=element_blank(),
        axis.text.y = element_text(angle = 90, vjust = 0.5, hjust=0.5)) +
  scale_y_continuous(breaks = seq(450000,700000, length.out = 6), labels = paste(seq(450000,700000, length.out = 6)), expand = c(0,0), limits = c(450000, 700000)) +
  scale_x_continuous(expand = c(0,0), limits = c(250000, 500000))
ggsave(paste0("plots/", elemname, "_predmap.png"), width = 128, height = 225, units = "mm", type = "cairo", dpi = 300, scale = 1.375)


# Aleatoric and Epistemic Uncertainty Maps

In [ ]:
#Remove variables to free up memory space
import gc
del elev_ann, elev_ann_test, elev_ann_train, elev_ann_val, elev_sd, evi_ann, evi_ann_test, evi_ann_train, evi_ann_val, evi_sd, grav_ann, grav_ann_test, grav_ann_train, grav_ann_val, grav_sd, loc, loc_test, loc_train, loc_val, locmean, locsd, x_test, x_train, x_val, y_test, y_train, y_val, y_pred, predraster, predloc
gc.collect()

In [ ]:
#Create Uncertainty Grid (have to aggregate to double cell size due to memory error)
predgrid_uncer = predgrid

In [ ]:
#Create list for repeated predictions
predictions_list = []

#Set Seed
tf.random.set_seed(2)

#Repeat predictions for full study area, and append to a list
for _ in range(100):
    predictions = model.predict([predimgs_elevann,predimgs_gravann,predimgs_eviann,predloc_ann], batch_size=batch_size)
    predictions_list.append(predictions)

# Convert to numpy array
predictions_array = np.array(predictions_list)

# Calculate the standard deviation for each data point, which we can attribute to epistemic uncertainty
epistemic = np.std(predictions_array, axis=0)

In [ ]:
#Add epistemic and aleatoric uncertainties to grid
predgrid_uncer['Epistemic'] = epistemic
predgrid_uncer['Aleatoric']= aleatoric

predgrid_uncer = predgrid_uncer[['x','y','Epistemic','Aleatoric']]


In [ ]:
%%R -i predgrid_uncer,logtrans,elemname
#Plot both aleatoric and epistemic uncertainties

ggplot(predgrid_uncer) + geom_raster(aes(x = x, y = y, fill = Aleatoric, interpolate = FALSE)) + theme_bw() + coord_equal() +
  scale_fill_viridis_c(option = "B", name = if(logtrans == TRUE){paste0("log(",elemname," Concentration)\nAleatoric Variance")}else{paste(elemname, " Concentration\nAleatoric Std")}) + labs(x = "Easting (metres BNG)", y = "Northing (metres BNG)") +
  theme(legend.justification = c(0, 0.5), legend.position = c(0.11, 0.85), legend.background=element_blank(),
        axis.text.y = element_text(angle = 90, vjust = 0.5, hjust=0.5))  +
  scale_y_continuous(breaks = seq(450000,700000, length.out = 6), labels = paste(seq(450000,700000, length.out = 6)), expand = c(0,0), limits = c(450000, 675000)) +
  scale_x_continuous(expand = c(0,0), limits = c(250000, 475000),breaks = seq(250000,500000, length.out = 6), labels = paste(seq(250000,500000, length.out = 6)))
ggsave(paste0("plots/", elemname, "_Aleatoric.png"), width = 128, height = 225, units = "mm", type = "cairo", dpi = 300, scale = 1.375)


ggplot(predgrid_uncer) + geom_raster(aes(x = x, y = y, fill = Epistemic, interpolate = FALSE)) + theme_bw() + coord_equal() +
  scale_fill_viridis_c(option = "B", name = if(logtrans == TRUE){paste0("log(",elemname," Concentration)\nEpistemic Std")}else{paste(elemname, " Concentration\nEpistemic Std")}) + labs(x = "Easting (metres BNG)", y = "Northing (metres BNG)") +
  theme(legend.justification = c(0, 0.5), legend.position = c(0.11, 0.85), legend.background=element_blank(),
        axis.text.y = element_text(angle = 90, vjust = 0.5, hjust=0.5))  +
  scale_y_continuous(breaks = seq(450000,700000, length.out = 6), labels = paste(seq(450000,700000, length.out = 6)), expand = c(0,0), limits = c(450000, 675000)) +
  scale_x_continuous(expand = c(0,0), limits = c(250000, 475000),breaks = seq(250000,500000, length.out = 6), labels = paste(seq(250000,500000, length.out = 6)))
ggsave(paste0("plots/", elemname, "_Epistemic.png"), width = 128, height = 225, units = "mm", type = "cairo", dpi = 300, scale = 1.375)